needed to reload modules, otherwise you will have nan errors

In [7]:
%load_ext autoreload
%autoreload 2

### Data

In [8]:
from data_process_and_get_mask import create_mask
from get_data import pbmc_train, pbmc_val

tf_list = [
    'IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A','STAT5B', 'STAT6'
]
#["GTF2I","GTF3A","NRF1","ELF1","STAT1","STAT2","IRF9","STAT3","STAT4","STAT5A","STAT5B","IRF3","IRF7","IRF1","IRF5","IRF8"]

mask, train, valid = create_mask(pbmc_train, pbmc_val, tf_list)
print("Printing mask:")
print(mask)
print("\nPrinting train:")
print(train)
print("\nPrinting valid:")
print(valid)

Printing mask:
shape: (363, 18)
┌─────────┬────────┬────────┬───────┬───┬──────┬──────┬──────┬───────────────┐
│ target  ┆ STAT5A ┆ STAT5B ┆ STAT4 ┆ … ┆ IRF7 ┆ IRF9 ┆ IRF6 ┆ unannotated_1 │
│ ---     ┆ ---    ┆ ---    ┆ ---   ┆   ┆ ---  ┆ ---  ┆ ---  ┆ ---           │
│ str     ┆ f64    ┆ f64    ┆ f64   ┆   ┆ f64  ┆ f64  ┆ f64  ┆ i32           │
╞═════════╪════════╪════════╪═══════╪═══╪══════╪══════╪══════╪═══════════════╡
│ IL2     ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ IRF1    ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ CCND1   ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ PRF1    ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ IFNG    ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0  ┆ 1.0  ┆ 0.0  ┆ 1             │
│ …       ┆ …      ┆ …      ┆ …     ┆ … ┆ …    ┆ …    ┆ …    ┆ …             │
│ ZC3HAV1 ┆ 0.0    ┆ 0.0    ┆ 0.0   ┆ … ┆ 0.0  ┆ 0.0  ┆ 0.0  ┆ 1             │
│ ZNF181  ┆ 0.0    ┆

In [9]:
from model.vega import VEGA
from model.decoder_default import DecoderVEGA
from model.encoder_with_mu_clamp import Encoder
from device import device

mask_np = mask.drop("target").to_numpy()

latent_dims= mask_np.shape[1]
input_dims = mask_np.shape[0]
dropout = 0.3
z_dropout = 0.5

vega = VEGA(
    encoder=Encoder(latent_dims, input_dims, dropout, z_dropout),
    decoder=DecoderVEGA(mask_np.T)
    ).to(device)

In [10]:
# model training
#vega, vega_losses, vega_klds, vega_mses = trainVEGA(vega, train, epochs = 20, beta = 0.0001) #takes about 2 mins on GPU # epoch 100

now use the common training functions

In [11]:
mask

target,STAT5A,STAT5B,STAT4,IRF1,STAT3,IRF2,STAT1,IRF8,STAT6,IRF4,IRF3,IRF5,STAT2,IRF7,IRF9,IRF6,unannotated_1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""IL2""",1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IRF1""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1
"""CCND1""",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""PRF1""",1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IFNG""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZC3HAV1""",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF181""",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF230""",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [12]:
from training.training import run_vega_model
run_vega_model(
    vega=vega,
    model_type="vega",
    train_data=train,
    valid_data=valid,
    mask_df=mask,
    epochs=5,
    N=5
)

epoch:  0  train_loss:  82.71822   valid_loss:  64.09644
epoch:  0  train_loss:  40.26143   valid_loss:  32.08836
epoch:  0  train_loss:  35.723995   valid_loss:  28.930416
epoch:  0  train_loss:  33.751404   valid_loss:  27.517126
epoch:  0  train_loss:  33.037632   valid_loss:  27.086815


(VEGA(
   (decoder): DecoderVEGA(
     (sparse_layer): Sequential(
       (0): SparseLayer()
     )
   )
   (encoder): Encoder(
     (encoder): Sequential(
       (0): Linear(in_features=363, out_features=800, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.3, inplace=False)
       (3): Linear(in_features=800, out_features=800, bias=True)
       (4): ReLU()
       (5): Dropout(p=0.3, inplace=False)
     )
     (mu): Sequential(
       (0): Linear(in_features=800, out_features=17, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
     (sigma): Sequential(
       (0): Linear(in_features=800, out_features=17, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
   )
 ),
            STAT5A    STAT5B     STAT4      IRF1     STAT3      IRF2     STAT1  \
 IL2      0.000053  0.000112  0.000000  0.000000 -0.000159  0.000000  0.000000   
 IRF1    -0.003727  0.003286 -0.002206  0.000897 -0.003300  0.000821  0.004303   
 CCND1    0.000041 -0.000011  0.000000  0.000254 -0.000